# Домашнее задание №1

В качестве данных я взяла отзывы с сайта [Фламп](https://moscow.flamp.ru/), а именно [отзывы на кофейни](https://moscow.flamp.ru/metarubric/kofejni). Здесь можно фильтровать отзывы по окраске ("положительные", "отрицательные")

In [102]:
import requests
from bs4 import BeautifulSoup
import random
from nltk import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import re
from collections import Counter

morph = MorphAnalyzer()
stopwords = stopwords.words('russian')

### Парсинг данных

Создаем два массива для положительных и отрицательных отзывов

In [120]:
positive_reviews = []
negative_reviews = []

Парсим сайт с отзывами

In [121]:
def get_from_url(href: str) -> BeautifulSoup:
    """
    Функция, которая парсит страницу с кофейнями
    :param href: строка со ссылкой
    :return: распарсенная страница
    """
    response = requests.get(href)
    html_content = response.text
    return BeautifulSoup(html_content, 'html.parser')

In [122]:
cafes = []
for page in range(1, 100):
    main_link = f"https://moscow.flamp.ru/metarubric/kofejni?page={page}"
    soup = get_from_url(main_link)
    cafes.extend(soup.find_all('a', {'class': 'card__link'}))

In [130]:
def get_reviews(href: str, rating: str = 'positive') -> list:
    """
    Функция, которая парсит страницу с отзывами
    :param href: строка со ссылкой
    :param rating: строка, по которой фильтруются отзывы {'positive', 'negative'}
    :return: список отзывов
    """
    soup = get_from_url(link + f'?rating={rating}')
    reviews = soup.find_all('p', {'class': 't-rich-text__p'})
    return [review.get_text().strip() for review in reviews]

In [131]:
for cafe in cafes:
    link = 'https:'+cafe['href']
    positive_reviews.extend(get_reviews(link, 'positive'))
    negative_reviews.extend(get_reviews(link, 'negative'))

Посмотрим, сколько отзывов мы достали и хватит ли нам их

In [132]:
len(positive_reviews), len(negative_reviews)

(6518, 6034)

В целом достаточно, но здесь имеются "лишние" отзывы. Например, первый отзыв это нераскрывшийся следующий, а между ними встряла кнопка 'Показать целиком':

In [134]:
negative_reviews[2:5]

['31 декабря в 10 утра зашли в кафе на ул. Арбат, 19. Быстро обслужили, спасибо. Далее выбрали место у окна - планировали посидеть долго, полюбоваться праздничной улицей. Столик не убран, попросила "Уберите, пожалуйста". Начался театр двух актеров. Чернявый юноша, который делал кофе, посмотрел на меня, что-то обсудил со светловолосой дамой в черной...',
 'Показать целиком',
 '31 декабря в 10 утра зашли в кафе на ул. Арбат, 19. Быстро обслужили, спасибо. Далее выбрали место у окна - планировали посидеть долго, полюбоваться праздничной улицей. Столик не убран, попросила "Уберите, пожалуйста". Начался театр двух актеров. Чернявый юноша, который делал кофе, посмотрел на меня, что-то обсудил со светловолосой дамой в черной шляпе (не продавец, видимо, менеджер) и сделал вид, что не услышал. Я просто смотрела на него, парень понял, что придется убрать. В кафе на тот момент было 4 посетителя, устать персонал не успел. Пара скрылась за дверью для персонала, по очереди выглядывала оттуда, парень

Для этого почистим отзывы и отберем только часть из них, чтобы уравнять количество

In [135]:
def remove_noise(reviews: list) -> list:
    """
    Функция, которая убирает лишние отзывы
    :param reviews: список отзывов
    :return: список отзывов
    """
    i = 0
    while i < len(reviews):
        if reviews[i] == 'Показать целиком':
            reviews.pop(i)
            reviews.pop(i-1)
            i -= 1
        else:
            i += 1
    return reviews

In [136]:
positive_reviews = remove_noise(positive_reviews)
negative_reviews = remove_noise(negative_reviews)

In [137]:
len(positive_reviews), len(negative_reviews)

(5140, 4704)

In [138]:
positive_reviews = positive_reviews[:4700]
negative_reviews = negative_reviews[:4700]

Отлично! Мы собрали отзывы, теперь можно с ними работать

### Классификация отзывов с помощью множеств

Для начала разделим выборку на трейн и тест

In [140]:
random.shuffle(positive_reviews)
random.shuffle(negative_reviews)
train_positive, test_positive = positive_reviews[:3700], positive_reviews[3700:]
train_negative, test_negative = negative_reviews[:3700], negative_reviews[3700:]

Теперь нужно токенизировать отзывы и собрать леммы для того, чтобы составить словари положительных и отрицательных слов

In [141]:
def tokenizer(text: str) -> list:
    """
    Функция, которая токенизирует строку текста
    :param text: строка текста
    :return: список токенов
    """
    return [morph.parse(word)[0].normal_form for word in word_tokenize(text.lower()) if (re.search(r"[^a-zа-я ]", word) is None) and word not in stopwords]

In [142]:
def tokenize_reviews(reviews: list) -> list:
    """
    Функция, которая сщбирает леммы из отзывов
    :param reviews: список отзывов
    :return: список лемм
    """
    lemmas = []
    for review in reviews:
        lemmas.extend(tokenizer(review))
    return lemmas

In [143]:
train_positive_lemmas = tokenize_reviews(train_positive)
train_negative_lemmas = tokenize_reviews(train_negative)

Выберем наиболее частотные слова для каждого класса

In [144]:
positive_words = Counter(train_positive_lemmas)
negative_words = Counter(train_negative_lemmas)

In [172]:
positive_words_freq = [k for k in positive_words if positive_words[k] > 5]
negative_words_freq = [k for k in negative_words if negative_words[k] > 5]

In [173]:
len(positive_words_freq), len(negative_words_freq)

(1520, 1621)

Выведем общее множество слов в отдельный массив и уберем слова из этого множества из словарей с положительными и отрицательными словами

In [174]:
both = []
for word in positive_words_freq:
    if word in negative_words_freq:
        both.append(word)

In [175]:
len(both)

1291

In [176]:
positive_words_freq = [word for word in positive_words_freq if word not in both]
negative_words_freq = [word for word in negative_words_freq if word not in both]

In [177]:
len(positive_words_freq), len(negative_words_freq)

(229, 330)

Теперь напишем функцию для классификации и функцию с метрикой

In [156]:
def check_rating(review: list) -> str:
    """
    Функция, которая классифицирует отзыв по наличи в нем слов из множеств положительных и отрицательных слов
    :param reviews: список отзывов
    :return: строка, которая говорит, к какому классу мы отнесли отзыв {'positive', 'negative', 'cannot define'}
    """
    positive = 0
    negative = 0
    for word in review:
        if word in positive_words_freq:
            positive += 1
        elif word in negative_words_freq:
            negative += 1
    if positive > negative:
        return 'positive'
    elif negative > positive:
        return 'negative'
    else:
        return 'cannot define'

In [157]:
def count_accuracy(positive, negative):
    posititve_lemmas = [tokenizer(review) for review in positive]
    negative_lemmas = [tokenizer(review) for review in negative]
    positive_ratings = [check_rating(review) for review in posititve_lemmas]
    negative_ratings = [check_rating(review) for review in negative_lemmas]
    print(f"accuracy = {(positive_ratings.count('positive') + negative_ratings.count('negative')) / (len(positive_ratings) + len(negative_ratings))}")

Проверим качество на трейне и на тесте

In [178]:
count_accuracy(train_positive, train_negative)

accuracy = 0.2875675675675676


In [179]:
count_accuracy(test_positive, test_negative)

accuracy = 0.2075


### Улучшения

Как видно, классификатор оказался очень плохим даже на трейне.
1. Мы никак не учитывали частоты. Мы просто создали два множества слов, определив порог попадания, но какое-то слово могло попасть в оба множества, при этом его частотность для положительного класса гораздо выше, чем для отрицательного, например, но мы не считаем это слово как индикатор тональности, потому что оно попало в оба класса.
2. Лучше здесь, конечно, использовать ML. Обучить классификатор (у нас просто бинарная классификация) на bag-of-words/tf-idf